In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')

In [14]:
from utils.deepmatcher_utils import wrapDm
import deepmatcher as dm
import pandas as pd
import numpy as np

In [23]:
test = pd.read_csv('../../datasets/Structured/Amazon-Google/merged_test.csv')
test_neg = test[test.label==0].copy()

In [24]:
test_neg.head(10)

,label,ltable_title,ltable_manufacturer,ltable_price,rtable_title,rtable_manufacturer,rtable_price,id
0,0,microsoft visual studio test agent 2005 cd 1 p...,microsoft software,5099.00,individual software professor teaches microsof...,NaN,29.99,9167
1,0,microsoft office excel 2007,microsoft,229.95,individual software professor teaches microsof...,NaN,29.99,9168
2,0,microsoft office small business 2007,microsoft software,735.33,individual software professor teaches microsof...,NaN,29.99,9169
3,0,microsoft visual studio test agent 2005 cd 1 p...,microsoft software,5099.00,122-00411 microsoft visual studio 2005 team ed...,NaN,2184.73,9170
4,0,microsoft visual studio professional w/msdn pr...,microsoft software,799.00,122-00411 microsoft visual studio 2005 team ed...,NaN,2184.73,9171
5,0,adobe dreamweaver cs3 [ mac ],adobe,399.00,adobe cs3 web premium,NaN,1659.99,9172
6,0,adobe flash pro cs3 upgrade [ mac ],adobe,199.00,adobe cs3 web premium,NaN,1659.99,9173
7,0,adobe creative suite cs3 web standard [ mac ],adobe,999.00,adobe cs3 web premium,NaN,1659.99,9174
8,0,adobe creative suite cs3 production premium up...,adobe,799.00,adobe cs3 web premium,NaN,1659.99,9175
9,0,adobe creative suite cs3 design premium upsell,adobe,1599.00,adobe cs3 web premium,NaN,1659.99,9176


In [13]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('../../models/amazongoogle_hybrid.pth')
model = model.to('cuda')

In [15]:
pred = wrapDm(test_neg,model)
np.count_nonzero(np.argmax(pred,axis=1)==0)

1982

In [9]:
def dropTokens(attr,tokensL):
    attr_tokens = list(map(lambda t:t.lower(),attr.split()))
    filtered_tokens = []
    for tok in attr_tokens:
        if tok not in tokensL:
            filtered_tokens.append(tok)
    return " ".join(filtered_tokens)

In [20]:
test_neg['ltable_title'] = test_neg['ltable_title'].apply(lambda attr:dropTokens(attr,['adobe','microsoft']))
test_neg['rtable_title'] = test_neg['rtable_title'].apply(lambda attr:dropTokens(attr,['adobe','microsoft']))

In [21]:
new_pred = wrapDm(test_neg,model)
np.count_nonzero(np.argmax(pred,axis=1)==0)

1982

In [22]:
test_neg.head(10)

,label,ltable_title,ltable_manufacturer,ltable_price,rtable_title,rtable_manufacturer,rtable_price,id
0,0,visual studio test agent 2005 cd 1 processor l...,microsoft software,5099.00,visual studio test agent 2005 cd 1 processor l...,NaN,29.99,9167
1,0,office excel 2007,microsoft,229.95,office excel 2007,NaN,29.99,9168
2,0,office small business 2007,microsoft software,735.33,office small business 2007,NaN,29.99,9169
3,0,visual studio test agent 2005 cd 1 processor l...,microsoft software,5099.00,visual studio test agent 2005 cd 1 processor l...,NaN,2184.73,9170
4,0,visual studio professional w/msdn professional...,microsoft software,799.00,visual studio professional w/msdn professional...,NaN,2184.73,9171
5,0,dreamweaver cs3 [ mac ],adobe,399.00,dreamweaver cs3 [ mac ],NaN,1659.99,9172
6,0,flash pro cs3 upgrade [ mac ],adobe,199.00,flash pro cs3 upgrade [ mac ],NaN,1659.99,9173
7,0,creative suite cs3 web standard [ mac ],adobe,999.00,creative suite cs3 web standard [ mac ],NaN,1659.99,9174
8,0,creative suite cs3 production premium upgrade,adobe,799.00,creative suite cs3 production premium upgrade,NaN,1659.99,9175
9,0,creative suite cs3 design premium upsell,adobe,1599.00,creative suite cs3 design premium upsell,NaN,1659.99,9176
